In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn import datasets 
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('wpbc.data',header=None)

In [6]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
1,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
2,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
3,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
4,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0


## Class labels in Column 1

In [3]:
y=data[1]

In [7]:
# N, R are the two classes
y.replace('N',1,inplace=True)
y.replace('R',0,inplace=True)

In [8]:
y

0      1
1      1
2      1
3      1
4      0
      ..
193    1
194    1
195    1
196    0
197    1
Name: 1, Length: 198, dtype: int64

In [ ]:
#Slicing to keep the columns that have a meaning

x=data.iloc[:,2:]

In [11]:
# the following is a ?, i.e. a missing value
#x.iloc[196][34]
#replace it with a numpy nan
x=x.replace('?',np.NaN)

#x[34].loc[196]=x[34].median()
x[34].fillna(x[34].median(),inplace=True)


# make all columns with 0-mean, and 1-std
x = preprocessing.scale(x)

In [15]:
x

array([[-0.45765783,  0.19268806,  1.24176957, ..., -0.45922582,
         1.11353031,  0.33834144],
       [ 0.41505085,  0.18317537, -2.77463023, ...,  1.32925886,
         0.07890337, -0.21530819],
       [ 2.01501676,  1.25493866, -1.12795296, ..., -0.00748783,
        -0.17975337, -0.58440794],
       ...,
       [-1.01037333, -0.07683821,  1.61262181, ..., -1.44414722,
         0.4410228 , -0.58440794],
       [-1.27218594,  1.27079314,  0.13154523, ..., -0.04821054,
         0.07890337, -0.39985806],
       [-1.18491507, -0.22587038,  1.36538698, ..., -0.49568685,
         0.3375601 , -0.58440794]])

In [17]:
#split thedat sets into training and tests
x_train, x_test, y_train, y_test = train_test_split (x,y, test_size=0.3, random_state=1)


In [27]:
x_train.shape

(138, 33)

In [31]:
type(x_train)

numpy.ndarray

In [28]:
x_test.shape

(60, 33)

In [18]:
#Define Neural Network
#logistic', the logistic sigmoid function,
#          returns f(x) = 1 / (1 + exp(-x))
# 'relu', the rectified linear unit function,
#         returns f(x) = max(0, x)

#  solver : {'lbfgs', 'sgd', 'adam'}, default='adam'
#
# batch_size: the size of the batch (number of data items used to compute error)
#
#early_stopping : bool, default=False
#    Whether to use early stopping to terminate training when validation
#    score is not improving. If set to true, it will automatically set
#    aside 10% of training data as validation and terminate training when
#    validation score is not improving by at least ``tol`` for
#    ``n_iter_no_change`` consecutive epochs.
#
clfANN = MLPClassifier(solver='adam', activation='relu',
                       batch_size=10,
                       tol=1e-7,
                       validation_fraction=0.2,
                       hidden_layer_sizes=(15,10,5,3), random_state=1, max_iter=50000, verbose=True)

## Here starts the training

In [ ]:
#train the classifiers
clfANN.fit(x_train, y_train)  

In [20]:
#get predictions on the train set
y_train_pred_ANN=clfANN.predict(x_train)

In [21]:
#test the trained model on the test set
y_test_pred_ANN=clfANN.predict(x_test)

In [22]:
confMatrixTestANN=confusion_matrix(y_test, y_test_pred_ANN, labels=None)
confMatrixTrainANN=confusion_matrix(y_train, y_train_pred_ANN, labels=None)

In [23]:
print ('\n Conf matrix, Train Set, Neural Net')
print (confMatrixTrainANN)
print ()


 Conf matrix, Train Set, Neural Net
[[ 32   0]
 [  0 106]]



In [32]:
# Confusion matrix upper row responds to 0 values R to our case and below to 1 valeu

In [25]:
print ('Conf matrix, Test Set, Neural Net')
print (confMatrixTestANN)
print ()

Conf matrix, Test Set, Neural Net
[[10  5]
 [ 8 37]]



In [26]:

# Measures of performance: Precision, Recall, F1


print ('Neural Net: Macro Precision, recall, f1-score')
print ( precision_recall_fscore_support(y_test, y_test_pred_ANN, average='macro'))
print ('Neural Net: Micro Precision, recall, f1-score')
print (precision_recall_fscore_support(y_test, y_test_pred_ANN, average='micro'))
print ('\n')


Neural Net: Macro Precision, recall, f1-score
(0.7182539682539683, 0.7444444444444445, 0.7283176593521421, None)
Neural Net: Micro Precision, recall, f1-score
(0.7833333333333333, 0.7833333333333333, 0.7833333333333333, None)


